<a href="https://colab.research.google.com/github/Cody9494/LEGALSKEPSIS-DATA/blob/main/LAWSKPEPSIS_STEP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

import pandas as pd

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from tqdm import tqdm

# 1. Φόρτωση df_chunks
df_chunks = pd.read_parquet("/content/drive/MyDrive/LAWSKEPSIS/df_chunks.parquet")

# 2. Φόρτωση μοντέλου και tokenizer
model_name = "intfloat/e5-mistral-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval().cuda()  # Χρησιμοποιούμε GPU

# 3. Συνάρτηση encoding (mean pooling)
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # (batch_size, seq_len, hidden_dim)
    attention_mask = attention_mask.to(token_embeddings.device)  # Move to same device
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
    return (token_embeddings * input_mask_expanded).sum(1) / input_mask_expanded.sum(1)


# 4. Δημιουργία embeddings
embeddings = []
batch_size = 16
texts = df_chunks["chunk_text"].tolist()
passages = [f"passage: {t.strip()}" for t in texts]

for i in tqdm(range(0, len(passages), batch_size)):
    batch = passages[i:i+batch_size]
    encoded = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=1024)
    with torch.no_grad():
        output = model(**{k: v.cuda() for k, v in encoded.items()})
    emb = mean_pooling(output, encoded['attention_mask'])
    embeddings.append(emb.cpu())

# 5. Ενοποίηση
all_embeddings = torch.cat(embeddings).numpy()
df_chunks["embedding"] = list(all_embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 3891/3891 [13:52:52<00:00, 12.84s/it]


In [ ]:
df_chunks.to_parquet("/content/drive/MyDrive/LAWSKEPSIS/df_chunks_with_embeddings.parquet", index=False)
print("✅ Saved embeddings with chunks.")

✅ Saved embeddings with chunks.
